# curvilinear grid from gridgen

In [ ]:
#need jupyterlab=3
%matplotlib widget
#%matplotlib inline

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
# minimal
import sys, time
import os, fnmatch, glob, shutil
from datetime import datetime, timedelta
# plot
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from matplotlib import image as mpimg
from ipywidgets import interact
from mpl_interactions import ipyplot as iplt
# map
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import shapely as shp
import shapely.plotting
import geopandas as gpd
from pyproj import Geod
# data
import numpy as np
import scipy as sp
import xarray as xr
import dask.array as da
#import xesmf as xe
import pandas as pd
import seaborn as sns
import zarr
from pydap.client import open_url
# extra plot
import holoviews as hv
import plotly.express as px
import geoviews as gv
import geoviews.feature as gf
hv.extension('bokeh')
gv.extension('bokeh')

In [ ]:
#GLOB-20KM

# Use WGS84 ellipsoid for accurate distance calculations
geod = Geod(ellps='WGS84')
grid_name='GLOB-20KM'
target_distance = 20000  # 20 km in meters
lat_south=-80
lat_north=80
lon_west=-179.99
lon_east=179.99
lon_range=abs(lon_west-lon_east)

In [ ]:
# ARC-12KM

# Use WGS84 ellipsoid for accurate distance calculations
geod = Geod(ellps='WGS84')
grid_name='ARC-12KM'
target_distance = 12000  # 20 km in meters
lat_south=66
lat_north=90
lon_west=-179.99
lon_east=179.99
lon_range=abs(lon_west-lon_east)

In [ ]:
# Calculate required number of latitude points for 20km spacing
def calculate_latitude_points():
    """Calculate number of latitude points needed for 20km spacing along meridians."""
    # Calculate distance between consecutive latitudes
    lat_points = []
    current_lat = lat_south
    lat_points.append(current_lat)

    while current_lat < lat_north:
        # Find next latitude that is exactly 20km north
        lon2, lat2, _ = geod.fwd(0, current_lat, 0, target_distance)  # Move north
        if current_lat > lat2:
            print('going too north closed to the pole')
            return np.array(lat_points)
        current_lat = lat2
        if current_lat <= lat_north:
            lat_points.append(current_lat)
        

    return np.array(lat_points)

# Calculate precise latitude points
latitudes = calculate_latitude_points()
n_lats = len(latitudes)
print(f"Number of latitude points: {n_lats}")
print(f"Latitude range: {latitudes[0]:.2f} to {latitudes[-1]:.2f}")

# Calculate longitude spacing for each latitude to maintain 20km resolution
lon_points_per_lat = []
lon_arrays = []

for i, lat in enumerate(latitudes):
    if abs(lat) < 89.9:  # Avoid poles where spacing becomes infinite
        # Calculate how many longitude points needed at this latitude
        # Distance for 1 degree of longitude at this latitude
        test_lon, test_lat, _ = geod.fwd(0, lat, 90, target_distance)
        degrees_for_20km = test_lon  # Since we started at longitude 0

        if degrees_for_20km > 0:
            n_lons_at_lat = int(lon_range / degrees_for_20km)
            # Ensure even number for symmetry
            n_lons_at_lat = n_lons_at_lat + (n_lons_at_lat % 2)
        else:
            n_lons_at_lat = 1  # At poles
    else:
        n_lons_at_lat = 1  # Pole points

    lon_points_per_lat.append(n_lons_at_lat)

    # Create longitude array for this latitude
    if n_lons_at_lat > 1:
        lons = np.linspace(lon_west, lon_east, n_lons_at_lat, endpoint=False)
    else:
        lons = np.array([0.0])  # Single point at poles

    lon_arrays.append(lons)
    
# Create 2D arrays with variable longitude resolution
max_lons = max(lon_points_per_lat)

# Initialize arrays with NaN
lon_2d = np.full((n_lats, max_lons), np.nan)
lat_2d = np.full((n_lats, max_lons), np.nan)

for i, (lat, lons) in enumerate(zip(latitudes, lon_arrays)):
    n = len(lons)
    lon_2d[i, :n] = lons
    lat_2d[i, :n] = lat

# Create a regular grid representation for easier handling
# Use the maximum number of longitude points for regular grid
regular_lons = np.linspace(lon_west, lon_east, max_lons, endpoint=False)
regular_lats = latitudes

lon_regular, lat_regular = np.meshgrid(regular_lons, regular_lats)

# Apply curvature to make it curvilinear using a suitable projection
projection = ccrs.Mollweide()  # Equal-area projection

# Transform to create curvilinear grid
proj_coords = projection.transform_points(ccrs.PlateCarree(), lon_regular, lat_regular)

# Create the dataset with both regular and variable resolution grids
ds = xr.Dataset(
    {
        'longitude': (('y', 'x'), lon_regular),
        'latitude': (('y', 'x'), lat_regular),
        'longitude_variable': (('y', 'x_var'), lon_2d),
        'latitude_variable': (('y', 'x_var'), lat_2d),
        'x_mollweide': (('y', 'x'), proj_coords[:, :, 0]),
        'y_mollweide': (('y', 'x'), proj_coords[:, :, 1]),
        'longitude_points_per_lat': (('y',), lon_points_per_lat)
    },
    coords={
        'x': np.arange(max_lons),
        'x_var': np.arange(max_lons),
        'y': np.arange(n_lats),
        'latitude_center': (('y',), latitudes)
    }
)

# Add comprehensive metadata
ds.attrs.update({
    'title': 'Global Geodesic Grid - Precise 20km Resolution',
    'resolution_meters': target_distance,
    'resolution_km': target_distance / 1000,
    'ellipsoid': 'WGS84',
    'grid_type': 'geodesic_curvilinear',
    'longitude_spacing': 'variable_by_latitude',
    'latitude_spacing': 'constant_20km'
})

In [ ]:
ds

In [ ]:
# Convert the longitude grid to a DataFrame
lon_df = pd.DataFrame(ds.longitude)

# Write the longitude grid DataFrame to a CSV file
lon_df.to_csv(grid_name+'.lon', index=False, header=False, sep=' ')

# Convert the latitude grid to a DataFrame
lat_df = pd.DataFrame(ds.latitude)

# Write the latitude grid DataFrame to a CSV file
lat_df.to_csv(grid_name+'.lat', index=False, header=False, sep=' ')